In [ ]:
# Cell 1: Graphical User Interface
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import lightgbm as lgb
import catboost as cbt
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt


def run_boost_algorithm():
    # Load the data set
    df = pd.read_csv(r"C:\Users\evanm\Desktop\CICIDS2017_sample_km.csv")

    # Define the feature and target variables
    X = df.drop(["Label"], axis=1)
    y = df["Label"]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Get the selected boost algorithm from the dropdown menu
    selected_boost = boost_menu.get()

    # Train the selected boost algorithm on the training data
    if selected_boost == "LightGBM":
        model = lgb.LGBMClassifier()
        model.fit(X_train, y_train)
    elif selected_boost == "XGBoost":
        model = xgb.XGBClassifier()
        model.fit(X_train, y_train)
    elif selected_boost == "CatBoost":
        model = cbt.CatBoostClassifier(verbose=0)
        model.fit(X_train, y_train)

    # Make predictions on the test data using the trained model
    y_pred = model.predict(X_test)

    # Generate a classification report and confusion matrix
    report = classification_report(y_test, y_pred)
    matrix = confusion_matrix(y_test, y_pred)

    # Display the report and matrix in a pop-up window
    messagebox.showinfo("Classification Report", report)
    plt.figure(figsize=(6, 6))
    sns.heatmap(matrix, annot=True, cmap="Blues")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()


# Create the GUI window using Tkinter
root = tk.Tk()
root.geometry("400x200")
root.title("LCCDE Security System")

# Add a dropdown menu for selecting the boost algorithm
boost_menu = ttk.Combobox(root, values=["LightGBM", "XGBoost", "CatBoost"])
boost_menu.set("Select Boost Algorithm")
boost_menu.pack(pady=10)

# Add a run button for running the selected boost algorithm on the data set
run_button = tk.Button(root, text="Run Boost Algorithm", command=run_boost_algorithm)
run_button.pack(pady=10)

# Start the GUI event loop
root.mainloop()

In [ ]:
# Cell 2: Algorithm modification for heightened certainty calculations
from sklearn.model_selection import RandomizedSearchCV

lg = LGBMClassifier(objective='multiclass', random_state=42)

param_dist_lg = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30]
}

random_search_lg = RandomizedSearchCV(lg, param_distributions=param_dist_lg, scoring='f1_weighted', n_iter=10, cv=3, n_jobs=-1, verbose=2)
random_search_lg.fit(X_train, y_train)

best_lg = random_search_lg.bestestimator
y_pred = best_lg.predict(X_test)

# LightGBM evaluation
y_pred_lg = best_lg.predict(X_test)
print(classification_report(y_test, y_pred_lg))
print("Accuracy of LightGBM: " + str(accuracy_score(y_test, y_pred_lg)))
print("Precision of LightGBM: " + str(precision_score(y_test, y_pred_lg, average='weighted')))
print("Recall of LightGBM: " + str(recall_score(y_test, y_pred_lg, average='weighted')))
print("Average F1 of LightGBM: " + str(f1_score(y_test, y_pred_lg, average='weighted')))
print("F1 of LightGBM for each type of attack: " + str(f1_score(y_test, y_pred_lg, average=None)))
lg_f1 = f1_score(y_test, y_pred_lg, average=None)

# Plot the confusion matrix
cm = confusion_matrix(y_test, y_pred_lg)
f, ax = plt.subplots(figsize=(5, 5))
sns.heatmap(cm, annot=True, linewidth=0.5, linecolor="red", fmt=".0f", ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()